# Basic Portfolio Analysis

#### Notebook: Initial Descriptive Analysis


In [13]:
# standard python libraries
import json
import datetime
import pandas as pd
import numpy as np
import glob
import os

# library for getting historical stock price data
import fastquant as fq

# Part 1: Collecting the Data

### INSERT DESCRIPTION

In [27]:
stocks_df = pd.DataFrame()

for file in glob.glob('data/20230315/stocks/*'):
    stock_df = pd.read_csv(file, index_col=[0])
    stock_name = os.path.basename(file)[:-4]
    stock_df.rename({'Price':stock_name}, axis=1, inplace=True)
    stock_df = stock_df[[stock_name]].copy()
    if stocks_df.empty:
        stocks_df = stock_df.copy()
    else:
        stocks_df = stocks_df.join(stock_df)

In [28]:
stocks_df

,AREIT,MER,CNVRG,SECB,DNL,MBT,ICT,BPI,CREIT,FPH,RCR,SMC2I,GLO,CNPF,TEL,AEV
Date,,,,,,,,,,,,,,,,
03/15/2023,34.45,315.2,15.00,93.05,7.77,57.00,197.7,104.30,2.45,61.85,5.59,74.8,"1,758.00",24.50,"1,310.00",49.35
03/14/2023,34.95,303.0,14.70,93.00,7.50,57.00,199.9,103.80,2.42,61.85,5.58,NaN,"1,777.00",25.00,"1,278.00",50.00
03/13/2023,34.40,317.8,15.04,94.10,7.69,59.40,207.4,105.10,2.42,61.95,5.58,74.8,"1,800.00",25.60,"1,278.00",52.50
03/10/2023,34.80,320.0,15.50,99.00,7.79,59.70,213.0,109.00,2.42,62.20,5.60,74.8,"1,820.00",25.45,"1,316.00",50.30
03/09/2023,34.95,314.0,15.72,101.00,7.86,60.10,212.4,108.00,2.45,61.95,5.65,74.8,"1,800.00",26.00,"1,280.00",51.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
08/20/2020,25.90,270.0,NaN,96.03,5.05,34.70,108.0,63.50,NaN,60.65,NaN,78.3,"2,112.00",16.20,"1,368.00",49.80
08/19/2020,25.95,271.0,NaN,97.52,5.10,36.00,108.1,62.50,NaN,60.00,NaN,78.3,"2,130.00",16.36,"1,380.00",49.60
08/18/2020,25.80,280.0,NaN,97.02,5.00,36.10,110.0,65.70,NaN,58.70,NaN,NaN,"2,130.00",16.20,"1,360.00",51.60


# Part 2: Data Preprocessing

## 2.1 Converting prices to returns

#### Many models are approaches used for time series modelling require the time series to be stationary. Stationarity assumes that the statistics of a process do not change over times. Using that assumption, we can build models that aim to forecast the future value of the process.

#### However, asset prices are usually non-stationary. Their statistic not only change over time, but we can also observe some trends (general patterns over time) or seasonality (patterns repeating over fixed time intervals). By transforming the prices into returns, we attempt to make the time series stationary. 

#### Another benefits of using returns, as opposed to prices, is normalization. It means that we can easily compare various return series, which would not be that simple with raw staock prices.

##### There are two types of returns.
<ol>
<li>Simple Returns
<li>Log Returns
</ol>
 